## Tuning di merging lists
Usa classe del prof per fsare recommender, è di hybridV4


In [1]:
import sys
sys.path.insert(0, './Lab/')
sys.path.insert(0, './New_Hybrids/')

from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from UserIcmKNNCFRecommender import UserIcmKNNCFRecommender
from ItemIcmKNNCFRecommender import ItemIcmKNNCFRecommender
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from GraphBased.P3alphaRecommender import P3alphaRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender
from MatrixFactorization.IALSRecommender import IALSRecommender
import matplotlib.pyplot as plt
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from WeightedHybridV3V4forBayesianSearch import WeightedHybridV3V4forBayesianSearch
from FeatureWeighting.CFW_D_Similarity_Linalg import CFW_D_Similarity_Linalg
# from WeightedHybridV4forBayesianSearch import WeightedHybridV4forBayesianSearch
%matplotlib inline 

import numpy as np

In [2]:
sys.path.insert(0, './datasets/')
from load_tables import load_ICM_csr, load_URM_csr
ICM_all = load_ICM_csr()
URM_all = load_URM_csr()

In [3]:
from Notebooks_utils.evaluation_function import evaluate_algorithm
from Notebooks_utils.data_splitter import train_test_holdout
from Base.Evaluation.Evaluator import EvaluatorHoldout

URM_train, URM_test = train_test_holdout(URM_all, train_perc = 0.9)#diminuisco size del test 
URM_train, URM_train_validation = train_test_holdout(URM_train, train_perc = 0.85)#aumento size del train x avere piu generalizzazione

evaluator_validation = EvaluatorHoldout(URM_train_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

## TUNING
Runna questa sezione per fare tuning, ci metterà 3 giorni tipo
SOtto poi c'è la sezione per fare testing, basta mettere manualmente i pesi trovati nel file .txt scritto da bayesian skopt con maps piu alti

In [4]:
from skopt.space import Real, Integer, Categorical
from ParameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 1500
n_random_starts = int(n_cases*0.35)
metric_to_optimize = "MAP"

In [5]:
userknn = {}
userknn["topK"] = 102
userknn["shrink"] = 1

itemknn = {}
itemknn["topK"] = 141
itemknn["shrink"] = 47

cy = {}
cy["epochs"] = 254
cy["topK"] = 881
cy["positive_threshold_BPR"] = 0.9765
cy["learning_rate"] = 0.0002
cy["batch_size"] = 10
cy["sgd_mode"] = "sdg"

uicm = {}
uicm["topK"] = 181
uicm["shrink"] = 0.1
uicm["normalize"] = True

iicm = {}
iicm["topK"] = 893
iicm["shrink"] = 2
iicm["normalize"] = True

alpha = {}
alpha["alpha"] = 0.547615508822564
alpha["topK"] = 500

beta= {}
beta["topK"]=500
beta["alpha"]=0.3784740936494376
beta["beta"]=0.1
beta["implicit"]=False
beta["normalize_similarity"]=False

ials={}
ials["epochs"]=45
ials["num_factors"]=50
ials["alpha"]=10.0
ials["epsilon"]=10.0
ials["reg"]=1e-07
ials["init_mean"]=1.0
ials["init_std"]=0.6773548197826565

hybridv2={}
#bayesianSearchhybridV2
#recommenders usati: [alpha,beta,userknn,uicm,cy,itemknn,iicm]
hybridv2["w0"]=0.001
hybridv2["w1"]=0.5593506236737317
hybridv2["w2"]=0.23267769028384777
hybridv2["w3"]=0.10077656126622785
hybridv2["w4"]=0.0001
hybridv2["w5"]=0.0001
hybridv2["w6"]=0.8397993296794579

itemcbf={}
itemcbf["topK"]=107
itemcbf["shrink"]=18.384308506219117
itemcbf["normalize"] = True
itemcbf["feature_weighting"] = 'BM25'
itemcbf["similarity"] = 'jaccard'



cfw_icm =  {}
cfw_icm["topK"] = 702
cfw_icm["add_zeros_quota"] = 0.975154924862614
cfw_icm['normalize_similarity'] = False

cfw_knn =  {}
cfw_knn["topK"] = 128
cfw_knn["add_zeros_quota"] = 0.4586460222330323
cfw_knn['normalize_similarity'] = False


# 6 recomm
fits = [alpha,beta,cy,iicm,ials,itemcbf,cfw_icm,cfw_knn]


recs = [
    P3alphaRecommender,
    RP3betaRecommender,
    #UserKNNCFRecommender,
    #UserIcmKNNCFRecommender,
    SLIM_BPR_Cython,
    #ItemKNNCFRecommender,
    ItemIcmKNNCFRecommender,
    IALSRecommender,
    ItemKNNCBFRecommender,
    CFW_D_Similarity_Linalg,
    CFW_D_Similarity_Linalg
    ]



only_URM = {
        "URM_train" : URM_train
    }

cython = {
        "URM_train" : URM_train,
        "recompile_cython" : False,
        "verbose" : False
    }

also_ICM = {
        "URM_train" : URM_train,
        "ICM" : ICM_all
    }

CBF = {
        "URM_train" : URM_train,
        "ICM_train" : ICM_all
    }
iknn = ItemIcmKNNCFRecommender(URM_train, ICM_all)
iknn.fit(**iicm)
cfw1  =  {
        "URM_train" : URM_train,
        "ICM" : ICM_all,
        "S_matrix_target":iknn.W_sparse.copy()
    }
iknn = ItemKNNCFRecommender(URM_train)
iknn.fit(**itemknn)
cfw2  =  {
        "URM_train" : URM_train,
        "ICM" : ICM_all,
        "S_matrix_target":iknn.W_sparse.copy()
    }
inits = [
        only_URM,only_URM,cython,also_ICM,only_URM,CBF,cfw1,cfw2
    ]

ItemKNNCFRecommender: URM Detected 89 (1.12 %) cold users.
ItemKNNCFRecommender: URM Detected 2687 (10.34 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
Similarity column 25975 ( 100 % ), 2701.25 column/sec, elapsed time 0.16 min
Similarity column 25975 ( 100 % ), 1304.66 column/sec, elapsed time 0.33 min
ItemKNNCFRecommender: URM Detected 89 (1.12 %) cold users.
ItemKNNCFRecommender: URM Detected 2687 (10.34 %) cold items.
Similarity column 25975 ( 100 % ), 2447.86 column/sec, elapsed time 0.18 min


In [6]:
recsToFit = []
for rec, init in zip(recs, inits):
    recsToFit.append(rec(**init))
for rec, fit in zip(recsToFit, fits):
    rec.fit(**fit)

P3alphaRecommender: URM Detected 89 (1.12 %) cold users.
P3alphaRecommender: URM Detected 2687 (10.34 %) cold items.
RP3betaRecommender: URM Detected 89 (1.12 %) cold users.
RP3betaRecommender: URM Detected 2687 (10.34 %) cold items.
ItemKNNCFRecommender: URM Detected 89 (1.12 %) cold users.
ItemKNNCFRecommender: URM Detected 2687 (10.34 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
IALSRecommender: URM Detected 89 (1.12 %) cold users.
IALSRecommender: URM Detected 2687 (10.34 %) cold items.
ItemKNNCBFRecommender: URM Detected 89 (1.12 %) cold users.
ItemKNNCBFRecommender: URM Detected 2687 (10.34 %) cold items.
CFW_D_Similarity_Linalg: URM Detected 89 (1.12 %) cold users.
CFW_D_Similarity_Linalg: URM Detected 2687 (10.34 %) cold items.
CFW_D_Similarity_Linalg: URM Detected 89 (1.12 %) cold users.
CFW_D_Similarity_Linalg: URM Detected 2687 (10.34 %) cold items.
Unable to read memory status: list index

SLIM_BPR_Recommender: Epoch 119 of 254. Elapsed time 2.71 sec
SLIM_BPR_Recommender: Epoch 120 of 254. Elapsed time 2.73 sec
SLIM_BPR_Recommender: Epoch 121 of 254. Elapsed time 2.75 sec
SLIM_BPR_Recommender: Epoch 122 of 254. Elapsed time 2.79 sec
SLIM_BPR_Recommender: Epoch 123 of 254. Elapsed time 2.81 sec
SLIM_BPR_Recommender: Epoch 124 of 254. Elapsed time 2.83 sec
SLIM_BPR_Recommender: Epoch 125 of 254. Elapsed time 2.86 sec
SLIM_BPR_Recommender: Epoch 126 of 254. Elapsed time 2.89 sec
SLIM_BPR_Recommender: Epoch 127 of 254. Elapsed time 2.91 sec
SLIM_BPR_Recommender: Epoch 128 of 254. Elapsed time 2.93 sec
SLIM_BPR_Recommender: Epoch 129 of 254. Elapsed time 2.96 sec
SLIM_BPR_Recommender: Epoch 130 of 254. Elapsed time 2.98 sec
SLIM_BPR_Recommender: Epoch 131 of 254. Elapsed time 3.01 sec
SLIM_BPR_Recommender: Epoch 132 of 254. Elapsed time 3.03 sec
SLIM_BPR_Recommender: Epoch 133 of 254. Elapsed time 3.06 sec
SLIM_BPR_Recommender: Epoch 134 of 254. Elapsed time 3.08 sec
SLIM_BPR

SLIM_BPR_Recommender: Epoch 252 of 254. Elapsed time 5.86 sec
SLIM_BPR_Recommender: Epoch 253 of 254. Elapsed time 5.88 sec
SLIM_BPR_Recommender: Epoch 254 of 254. Elapsed time 5.91 sec
SLIM_BPR_Recommender: Terminating at epoch 254. Elapsed time 46.88 sec
Deallocating Cython objects
Similarity column 25975 ( 100 % ), 4024.82 column/sec, elapsed time 0.11 min
Similarity column 25975 ( 100 % ), 1799.87 column/sec, elapsed time 0.24 min
IALSRecommender: Epoch 1 of 45. Elapsed time 6.69 sec
IALSRecommender: Epoch 2 of 45. Elapsed time 13.36 sec
IALSRecommender: Epoch 3 of 45. Elapsed time 22.01 sec
IALSRecommender: Epoch 4 of 45. Elapsed time 29.87 sec
IALSRecommender: Epoch 5 of 45. Elapsed time 37.20 sec
IALSRecommender: Epoch 6 of 45. Elapsed time 44.16 sec
IALSRecommender: Epoch 7 of 45. Elapsed time 53.68 sec
IALSRecommender: Epoch 8 of 45. Elapsed time 1.02 min
IALSRecommender: Epoch 9 of 45. Elapsed time 1.14 min
IALSRecommender: Epoch 10 of 45. Elapsed time 1.25 min
IALSRecommende

In [7]:
hyperparameters_range_dictionary = {}#forse qui fa il matto
hyperparameters_range_dictionary["w0"] = Real(0,10) #faccio da 0 a 1 real?
hyperparameters_range_dictionary["w1"] = Real(0,10)
hyperparameters_range_dictionary["w2"] = Real(0,10)
hyperparameters_range_dictionary["w3"] = Real(0,10)
hyperparameters_range_dictionary["w4"] = Real(0,10)
hyperparameters_range_dictionary["w5"] = Real(0,10)
hyperparameters_range_dictionary["w6"] = Real(0,10)
hyperparameters_range_dictionary["w7"] = Real(0,10)



recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train,ICM_all,recsToFit],   
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)


recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation,ICM_all],    
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

 

In [8]:
recommender_class=WeightedHybridV3V4forBayesianSearch

In [9]:
import time
start_time = time.time()
parameterSearch = SearchBayesianSkopt(recommender_class,
                                 evaluator_validation=evaluator_validation,
                                 evaluator_test=evaluator_test)


parameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       parameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path,
                       output_file_name_root = recommender_class.RECOMMENDER_NAME,
                       metric_to_optimize = metric_to_optimize
                      )

end_time = time.time()

print("Ha finito in questo tempo: "+str(end_time-start_time)+" secondi")

Iteration No: 1 started. Evaluating function at random point.
WeightedHybridV3V4forBayesianSearch: URM Detected 89 (1.12 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2687 (10.34 %) cold items.
SearchBayesianSkopt: Testing config: {'w0': 9.06579158513438, 'w1': 0.6495370495620157, 'w2': 1.5265100987434423, 'w3': 9.173924875654322, 'w4': 9.435741107031832, 'w5': 0.08034230427157564, 'w6': 7.356923939360249, 'w7': 0.7175461104221038}
--------FITTING...-------
EvaluatorHoldout: Processed 4722 ( 100.00% ) in 25.42 sec. Users per second: 186
SearchBayesianSkopt: New best config found. Config 0: {'w0': 9.06579158513438, 'w1': 0.6495370495620157, 'w2': 1.5265100987434423, 'w3': 9.173924875654322, 'w4': 9.435741107031832, 'w5': 0.08034230427157564, 'w6': 7.356923939360249, 'w7': 0.7175461104221038} - results: ROC_AUC: 0.1619072, PRECISION: 0.0321686, PRECISION_RECALL_MIN_DEN: 0.1369288, RECALL: 0.1347680, MAP: 0.0616875, MRR: 0.1148938, NDCG: 0.0919072, F1: 0.0519394, HIT_RA

EvaluatorHoldout: Processed 4722 ( 100.00% ) in 17.55 sec. Users per second: 269
SearchBayesianSkopt: Config 4 is suboptimal. Config: {'w0': 1.8553279246752588, 'w1': 6.48565481638572, 'w2': 1.179810905366592, 'w3': 3.461149650593583, 'w4': 1.9082628632111982, 'w5': 4.55754750950669, 'w6': 6.624907784570998, 'w7': 0.9734947190249557} - results: ROC_AUC: 0.1700409, PRECISION: 0.0343287, PRECISION_RECALL_MIN_DEN: 0.1498450, RECALL: 0.1475283, MAP: 0.0661648, MRR: 0.1206685, NDCG: 0.0986698, F1: 0.0556971, HIT_RATE: 0.3432867, ARHR: 0.1348681, NOVELTY: 0.0049423, AVERAGE_POPULARITY: 0.0876008, DIVERSITY_MEAN_INTER_LIST: 0.9905930, DIVERSITY_HERFINDAHL: 0.9990383, COVERAGE_ITEM: 0.4696824, COVERAGE_ITEM_CORRECT: 0.0411165, COVERAGE_USER: 0.5941865, COVERAGE_USER_CORRECT: 0.1615704, DIVERSITY_GINI: 0.1804676, SHANNON_ENTROPY: 12.0235768, 

Iteration No: 5 ended. Evaluation done at random point.
Time taken: 17.7062
Function value obtained: -0.0662
Current minimum: -0.0685
Iteration No: 6 sta

EvaluatorHoldout: Processed 4722 ( 100.00% ) in 16.48 sec. Users per second: 287
SearchBayesianSkopt: Config 10 is suboptimal. Config: {'w0': 2.2226433695384147, 'w1': 1.109776306783883, 'w2': 2.0123573881974726, 'w3': 2.831684158854699, 'w4': 0.493583692005486, 'w5': 4.016485744487568, 'w6': 6.932220990815749, 'w7': 8.032022765891723} - results: ROC_AUC: 0.1359281, PRECISION: 0.0275307, PRECISION_RECALL_MIN_DEN: 0.1195430, RECALL: 0.1177255, MAP: 0.0497209, MRR: 0.0927227, NDCG: 0.0764516, F1: 0.0446255, HIT_RATE: 0.2753071, ARHR: 0.1022599, NOVELTY: 0.0051060, AVERAGE_POPULARITY: 0.0474445, DIVERSITY_MEAN_INTER_LIST: 0.9963089, DIVERSITY_HERFINDAHL: 0.9996098, COVERAGE_ITEM: 0.5139172, COVERAGE_ITEM_CORRECT: 0.0369971, COVERAGE_USER: 0.5941865, COVERAGE_USER_CORRECT: 0.1338870, DIVERSITY_GINI: 0.2357561, SHANNON_ENTROPY: 12.7194252, 

Iteration No: 11 ended. Evaluation done at random point.
Time taken: 16.7433
Function value obtained: -0.0497
Current minimum: -0.0685
Iteration No: 12

EvaluatorHoldout: Processed 4722 ( 100.00% ) in 17.79 sec. Users per second: 265
SearchBayesianSkopt: Config 16 is suboptimal. Config: {'w0': 4.013991211566289, 'w1': 4.184035369276805, 'w2': 9.26616148682466, 'w3': 9.907908465037586, 'w4': 1.747873743067552, 'w5': 3.6339765562473096, 'w6': 9.46887538193577, 'w7': 3.47302450896876} - results: ROC_AUC: 0.1530282, PRECISION: 0.0301779, PRECISION_RECALL_MIN_DEN: 0.1303887, RECALL: 0.1283411, MAP: 0.0570714, MRR: 0.1064780, NDCG: 0.0858940, F1: 0.0488656, HIT_RATE: 0.3017789, ARHR: 0.1181005, NOVELTY: 0.0050207, AVERAGE_POPULARITY: 0.0647582, DIVERSITY_MEAN_INTER_LIST: 0.9943771, DIVERSITY_HERFINDAHL: 0.9994167, COVERAGE_ITEM: 0.5067180, COVERAGE_ITEM_CORRECT: 0.0375746, COVERAGE_USER: 0.5941865, COVERAGE_USER_CORRECT: 0.1443312, DIVERSITY_GINI: 0.2207920, SHANNON_ENTROPY: 12.4962601, 

Iteration No: 17 ended. Evaluation done at random point.
Time taken: 17.9709
Function value obtained: -0.0571
Current minimum: -0.0685
Iteration No: 18 sta

EvaluatorHoldout: Processed 4722 ( 100.00% ) in 15.42 sec. Users per second: 306
SearchBayesianSkopt: Config 22 is suboptimal. Config: {'w0': 5.28132368901757, 'w1': 2.978550514894827, 'w2': 4.303637873889377, 'w3': 1.1099627687071958, 'w4': 8.018602070226809, 'w5': 6.102479260260166, 'w6': 9.093646923020245, 'w7': 3.020905018781907} - results: ROC_AUC: 0.1728798, PRECISION: 0.0347946, PRECISION_RECALL_MIN_DEN: 0.1534337, RECALL: 0.1511761, MAP: 0.0669312, MRR: 0.1227698, NDCG: 0.1003556, F1: 0.0565692, HIT_RATE: 0.3479458, ARHR: 0.1367727, NOVELTY: 0.0048883, AVERAGE_POPULARITY: 0.0908649, DIVERSITY_MEAN_INTER_LIST: 0.9901093, DIVERSITY_HERFINDAHL: 0.9989900, COVERAGE_ITEM: 0.4373436, COVERAGE_ITEM_CORRECT: 0.0411550, COVERAGE_USER: 0.5941865, COVERAGE_USER_CORRECT: 0.1652196, DIVERSITY_GINI: 0.1584860, SHANNON_ENTROPY: 11.8445872, 

Iteration No: 23 ended. Evaluation done at random point.
Time taken: 15.5607
Function value obtained: -0.0669
Current minimum: -0.0685
Iteration No: 24 s

...

EvaluatorHoldout: Processed 4722 ( 100.00% ) in 24.98 sec. Users per second: 189
SearchBayesianSkopt: Config 1429 is suboptimal. Config: {'w0': 0.0, 'w1': 10.0, 'w2': 0.0, 'w3': 10.0, 'w4': 10.0, 'w5': 10.0, 'w6': 0.0, 'w7': 10.0} - results: ROC_AUC: 0.1740282, PRECISION: 0.0350064, PRECISION_RECALL_MIN_DEN: 0.1550632, RECALL: 0.1527929, MAP: 0.0681777, MRR: 0.1234185, NDCG: 0.1015666, F1: 0.0569621, HIT_RATE: 0.3500635, ARHR: 0.1377902, NOVELTY: 0.0048783, AVERAGE_POPULARITY: 0.0958018, DIVERSITY_MEAN_INTER_LIST: 0.9891167, DIVERSITY_HERFINDAHL: 0.9988907, COVERAGE_ITEM: 0.4377286, COVERAGE_ITEM_CORRECT: 0.0412705, COVERAGE_USER: 0.5941865, COVERAGE_USER_CORRECT: 0.1657229, DIVERSITY_GINI: 0.1556329, SHANNON_ENTROPY: 11.7686498, 

Iteration No: 1430 ended. Search finished for the next optimal point.
Time taken: 461.6074
Function value obtained: -0.0682
Current minimum: -0.0697
Iteration No: 1431 started. Searching for the next optimal point.
WeightedHybridV3V4forBayesianSearch: URM De

Iteration No: 1436 ended. Search finished for the next optimal point.
Time taken: 480.8599
Function value obtained: -0.0691
Current minimum: -0.0697
Iteration No: 1437 started. Searching for the next optimal point.
WeightedHybridV3V4forBayesianSearch: URM Detected 89 (1.12 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2687 (10.34 %) cold items.
SearchBayesianSkopt: Testing config: {'w0': 1.5849533073984101, 'w1': 5.657930051137906, 'w2': 0.0, 'w3': 6.72750556849604, 'w4': 6.8627248522516195, 'w5': 10.0, 'w6': 0.0, 'w7': 10.0}
--------FITTING...-------
EvaluatorHoldout: Processed 4722 ( 100.00% ) in 19.99 sec. Users per second: 236
SearchBayesianSkopt: Config 1436 is suboptimal. Config: {'w0': 1.5849533073984101, 'w1': 5.657930051137906, 'w2': 0.0, 'w3': 6.72750556849604, 'w4': 6.8627248522516195, 'w5': 10.0, 'w6': 0.0, 'w7': 10.0} - results: ROC_AUC: 0.1756047, PRECISION: 0.0356205, PRECISION_RECALL_MIN_DEN: 0.1579837, RECALL: 0.1556261, MAP: 0.0689332, MRR: 0.124528

EvaluatorHoldout: Processed 3000 ( 63.53% ) in 36.25 sec. Users per second: 83
EvaluatorHoldout: Processed 4722 ( 100.00% ) in 47.66 sec. Users per second: 99
SearchBayesianSkopt: Config 1442 is suboptimal. Config: {'w0': 3.1168620241212084, 'w1': 4.137225202891133, 'w2': 0.0, 'w3': 5.756927254603049, 'w4': 6.640733070057875, 'w5': 10.0, 'w6': 0.0, 'w7': 10.0} - results: ROC_AUC: 0.1766799, PRECISION: 0.0357687, PRECISION_RECALL_MIN_DEN: 0.1587005, RECALL: 0.1563632, MAP: 0.0691316, MRR: 0.1252786, NDCG: 0.1033558, F1: 0.0582195, HIT_RATE: 0.3576874, ARHR: 0.1395932, NOVELTY: 0.0049030, AVERAGE_POPULARITY: 0.0902730, DIVERSITY_MEAN_INTER_LIST: 0.9903711, DIVERSITY_HERFINDAHL: 0.9990161, COVERAGE_ITEM: 0.4445043, COVERAGE_ITEM_CORRECT: 0.0430029, COVERAGE_USER: 0.5941865, COVERAGE_USER_CORRECT: 0.1696238, DIVERSITY_GINI: 0.1627701, SHANNON_ENTROPY: 11.8875183, 

Iteration No: 1443 ended. Search finished for the next optimal point.
Time taken: 558.1396
Function value obtained: -0.0691
Cu

SearchBayesianSkopt: Config 1448 is suboptimal. Config: {'w0': 3.9544249305974755, 'w1': 3.490306576479819, 'w2': 10.0, 'w3': 5.829855239316688, 'w4': 6.946653501294572, 'w5': 10.0, 'w6': 0.0, 'w7': 10.0} - results: ROC_AUC: 0.1777821, PRECISION: 0.0357476, PRECISION_RECALL_MIN_DEN: 0.1586211, RECALL: 0.1563069, MAP: 0.0692766, MRR: 0.1253782, NDCG: 0.1034950, F1: 0.0581876, HIT_RATE: 0.3574756, ARHR: 0.1396873, NOVELTY: 0.0048879, AVERAGE_POPULARITY: 0.0929716, DIVERSITY_MEAN_INTER_LIST: 0.9897742, DIVERSITY_HERFINDAHL: 0.9989565, COVERAGE_ITEM: 0.4402310, COVERAGE_ITEM_CORRECT: 0.0426179, COVERAGE_USER: 0.5941865, COVERAGE_USER_CORRECT: 0.1696238, DIVERSITY_GINI: 0.1589375, SHANNON_ENTROPY: 11.8321380, 

Iteration No: 1449 ended. Search finished for the next optimal point.
Time taken: 1305.2538
Function value obtained: -0.0693
Current minimum: -0.0697
Iteration No: 1450 started. Searching for the next optimal point.
WeightedHybridV3V4forBayesianSearch: URM Detected 89 (1.12 %) cold u

KeyboardInterrupt: 

In [ ]:
from Base.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")
#search_metadata = data_loader.load_data("WeightedHybridV3V4forBayesianSearch_metadata.zip")
best_parameters = search_metadata["hyperparameters_best"]
print(best_parameters)



# CHECK performance
## L'ultimo qui e' quello con map mega alto su kaggle

In [5]:
#fa 3 train/test split e poi fa la media dei map per fare un po di "cross validation" e vedere su 3 quale metodo va meglio
URM_train, URM_test = train_test_holdout(URM_all, train_perc = 0.8)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

URM_train2, URM_test2 = train_test_holdout(URM_all, train_perc = 0.8)
evaluator_test2 = EvaluatorHoldout(URM_test2, cutoff_list=[10])

URM_train3, URM_test3 = train_test_holdout(URM_all, train_perc = 0.8)
evaluator_test3 = EvaluatorHoldout(URM_test3, cutoff_list=[10])

In [7]:
#devo rifittare con nuovo train/test split


userknn = {}
userknn["topK"] = 102
userknn["shrink"] = 1

itemknn = {}
itemknn["topK"] = 141
itemknn["shrink"] = 47

cy = {}
cy["epochs"] = 254
cy["topK"] = 881
cy["positive_threshold_BPR"] = 0.9765
cy["learning_rate"] = 0.0002
cy["batch_size"] = 10
cy["sgd_mode"] = "sdg"

uicm = {}
uicm["topK"] = 181
uicm["shrink"] = 0.1
uicm["normalize"] = True

iicm = {}
iicm["topK"] = 893
iicm["shrink"] = 2
iicm["normalize"] = True

alpha = {}
alpha["alpha"] = 0.547615508822564
alpha["topK"] = 500

beta= {}
beta["topK"]=500
beta["alpha"]=0.3784740936494376
beta["beta"]=0.1
beta["implicit"]=False
beta["normalize_similarity"]=False

ials={}
ials["epochs"]=45
ials["num_factors"]=50
ials["alpha"]=10.0
ials["epsilon"]=10.0
ials["reg"]=1e-07
ials["init_mean"]=1.0
ials["init_std"]=0.6773548197826565

hybridv2={}
#bayesianSearchhybridV2
#recommenders usati: [alpha,beta,userknn,uicm,cy,itemknn,iicm]
hybridv2["w0"]=0.001
hybridv2["w1"]=0.5593506236737317
hybridv2["w2"]=0.23267769028384777
hybridv2["w3"]=0.10077656126622785
hybridv2["w4"]=0.0001
hybridv2["w5"]=0.0001
hybridv2["w6"]=0.8397993296794579

itemcbf={}
itemcbf["topK"]=107
itemcbf["shrink"]=18.384308506219117
itemcbf["normalize"] = True
itemcbf["feature_weighting"] = 'BM25'
itemcbf["similarity"] = 'jaccard'

recs = [
    P3alphaRecommender,
    RP3betaRecommender,
    #UserKNNCFRecommender,
    #UserIcmKNNCFRecommender,
    SLIM_BPR_Cython,
    #ItemKNNCFRecommender,
    ItemIcmKNNCFRecommender,
    IALSRecommender,
    ItemKNNCBFRecommender
    ]



only_URM = {
        "URM_train" : URM_train
    }

cython = {
        "URM_train" : URM_train,
        "recompile_cython" : False,
        "verbose" : False
    }

also_ICM = {
        "URM_train" : URM_train,
        "ICM" : ICM_all
    }

CBF = {
        "URM_train" : URM_train,
        "ICM_train" : ICM_all
    }

inits = [
        only_URM,only_URM,cython,also_ICM,only_URM,CBF 
    ]

In [8]:
only_URM = {
        "URM_train" : URM_train2
    }

cython = {
        "URM_train" : URM_train2,
        "recompile_cython" : False,
        "verbose" : False
    }

also_ICM = {
        "URM_train" : URM_train2,
        "ICM" : ICM_all
    }

CBF = {
        "URM_train" : URM_train2,
        "ICM_train" : ICM_all
    }
inits2 = [
        only_URM,only_URM,cython,also_ICM,only_URM,CBF 
    ]

In [9]:
only_URM = {
        "URM_train" : URM_train3
    }

cython = {
        "URM_train" : URM_train3,
        "recompile_cython" : False,
        "verbose" : False
    }

also_ICM = {
        "URM_train" : URM_train3,
        "ICM" : ICM_all
    }

CBF = {
        "URM_train" : URM_train3,
        "ICM_train" : ICM_all
    }
inits3 = [
        only_URM,only_URM,cython,also_ICM,only_URM,CBF 
    ]

In [10]:
fits = [alpha,beta,cy,iicm,ials,itemcbf]

In [11]:
recsToFit = []
for rec, init in zip(recs, inits):
    recsToFit.append(rec(**init))
for rec, fit in zip(recsToFit, fits):
    rec.fit(**fit)

P3alphaRecommender: URM Detected 73 (0.92 %) cold users.
P3alphaRecommender: URM Detected 2431 (9.36 %) cold items.
RP3betaRecommender: URM Detected 73 (0.92 %) cold users.
RP3betaRecommender: URM Detected 2431 (9.36 %) cold items.
ItemKNNCFRecommender: URM Detected 73 (0.92 %) cold users.
ItemKNNCFRecommender: URM Detected 2431 (9.36 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
IALSRecommender: URM Detected 73 (0.92 %) cold users.
IALSRecommender: URM Detected 2431 (9.36 %) cold items.
ItemKNNCBFRecommender: URM Detected 73 (0.92 %) cold users.
ItemKNNCBFRecommender: URM Detected 2431 (9.36 %) cold items.
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Epoch 1 of 254. Elapsed time 0.03 sec
SLIM_BPR_Recommender: Epoch 2 of 254. Elapsed time 0.06 sec
SLIM_BPR_Recommender: Epoch 3 of 254. Elapsed time 0.11 sec
SLIM_BPR_Recommender: Epoch 4 of 254. Elapsed time 0.15 sec
SLIM_

SLIM_BPR_Recommender: Epoch 123 of 254. Elapsed time 3.16 sec
SLIM_BPR_Recommender: Epoch 124 of 254. Elapsed time 3.18 sec
SLIM_BPR_Recommender: Epoch 125 of 254. Elapsed time 3.20 sec
SLIM_BPR_Recommender: Epoch 126 of 254. Elapsed time 3.22 sec
SLIM_BPR_Recommender: Epoch 127 of 254. Elapsed time 3.24 sec
SLIM_BPR_Recommender: Epoch 128 of 254. Elapsed time 3.25 sec
SLIM_BPR_Recommender: Epoch 129 of 254. Elapsed time 3.27 sec
SLIM_BPR_Recommender: Epoch 130 of 254. Elapsed time 3.29 sec
SLIM_BPR_Recommender: Epoch 131 of 254. Elapsed time 3.31 sec
SLIM_BPR_Recommender: Epoch 132 of 254. Elapsed time 3.33 sec
SLIM_BPR_Recommender: Epoch 133 of 254. Elapsed time 3.35 sec
SLIM_BPR_Recommender: Epoch 134 of 254. Elapsed time 3.37 sec
SLIM_BPR_Recommender: Epoch 135 of 254. Elapsed time 3.39 sec
SLIM_BPR_Recommender: Epoch 136 of 254. Elapsed time 3.41 sec
SLIM_BPR_Recommender: Epoch 137 of 254. Elapsed time 3.43 sec
SLIM_BPR_Recommender: Epoch 138 of 254. Elapsed time 3.44 sec
SLIM_BPR

Deallocating Cython objects
Similarity column 25975 ( 100 % ), 4875.77 column/sec, elapsed time 0.09 min
Similarity column 25975 ( 100 % ), 2719.01 column/sec, elapsed time 0.16 min
IALSRecommender: Epoch 1 of 45. Elapsed time 5.72 sec
IALSRecommender: Epoch 2 of 45. Elapsed time 10.19 sec
IALSRecommender: Epoch 3 of 45. Elapsed time 14.60 sec
IALSRecommender: Epoch 4 of 45. Elapsed time 19.75 sec
IALSRecommender: Epoch 5 of 45. Elapsed time 23.91 sec
IALSRecommender: Epoch 6 of 45. Elapsed time 28.45 sec
IALSRecommender: Epoch 7 of 45. Elapsed time 33.72 sec
IALSRecommender: Epoch 8 of 45. Elapsed time 38.02 sec
IALSRecommender: Epoch 9 of 45. Elapsed time 42.18 sec
IALSRecommender: Epoch 10 of 45. Elapsed time 47.33 sec
IALSRecommender: Epoch 11 of 45. Elapsed time 52.10 sec
IALSRecommender: Epoch 12 of 45. Elapsed time 56.66 sec
IALSRecommender: Epoch 13 of 45. Elapsed time 1.03 min
IALSRecommender: Epoch 14 of 45. Elapsed time 1.11 min
IALSRecommender: Epoch 15 of 45. Elapsed time 

In [12]:
recsToFit2 = []
for rec, init in zip(recs, inits2):
    recsToFit2.append(rec(**init))
for rec, fit in zip(recsToFit2, fits):
    rec.fit(**fit)
    
#passo URM_train2 e recstoFit2

P3alphaRecommender: URM Detected 58 (0.73 %) cold users.
P3alphaRecommender: URM Detected 2418 (9.31 %) cold items.
RP3betaRecommender: URM Detected 58 (0.73 %) cold users.
RP3betaRecommender: URM Detected 2418 (9.31 %) cold items.
ItemKNNCFRecommender: URM Detected 58 (0.73 %) cold users.
ItemKNNCFRecommender: URM Detected 2418 (9.31 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
IALSRecommender: URM Detected 58 (0.73 %) cold users.
IALSRecommender: URM Detected 2418 (9.31 %) cold items.
ItemKNNCBFRecommender: URM Detected 58 (0.73 %) cold users.
ItemKNNCBFRecommender: URM Detected 2418 (9.31 %) cold items.
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Epoch 1 of 254. Elapsed time 0.02 sec
SLIM_BPR_Recommender: Epoch 2 of 254. Elapsed time 0.06 sec
SLIM_BPR_Recommender: Epoch 3 of 254. Elapsed time 0.10 sec
SLIM_BPR_Recommender: Epoch 4 of 254. Elapsed time 0.14 sec
SLIM_

SLIM_BPR_Recommender: Terminating at epoch 254. Elapsed time 30.47 sec
Deallocating Cython objects
Similarity column 25975 ( 100 % ), 4735.01 column/sec, elapsed time 0.09 min
Similarity column 25975 ( 100 % ), 2614.63 column/sec, elapsed time 0.17 min
IALSRecommender: Epoch 1 of 45. Elapsed time 4.88 sec
IALSRecommender: Epoch 2 of 45. Elapsed time 9.78 sec
IALSRecommender: Epoch 3 of 45. Elapsed time 13.95 sec
IALSRecommender: Epoch 4 of 45. Elapsed time 18.50 sec
IALSRecommender: Epoch 5 of 45. Elapsed time 23.63 sec
IALSRecommender: Epoch 6 of 45. Elapsed time 27.81 sec
IALSRecommender: Epoch 7 of 45. Elapsed time 32.17 sec
IALSRecommender: Epoch 8 of 45. Elapsed time 37.30 sec
IALSRecommender: Epoch 9 of 45. Elapsed time 41.75 sec
IALSRecommender: Epoch 10 of 45. Elapsed time 46.34 sec
IALSRecommender: Epoch 11 of 45. Elapsed time 50.99 sec
IALSRecommender: Epoch 12 of 45. Elapsed time 55.58 sec
IALSRecommender: Epoch 13 of 45. Elapsed time 1.01 min
IALSRecommender: Epoch 14 of 45

In [13]:
recsToFit3 = []
for rec, init in zip(recs, inits3):
    recsToFit3.append(rec(**init))
for rec, fit in zip(recsToFit3, fits):
    rec.fit(**fit)
    
#passo URM_train2 e recstoFit2

P3alphaRecommender: URM Detected 67 (0.84 %) cold users.
P3alphaRecommender: URM Detected 2429 (9.35 %) cold items.
RP3betaRecommender: URM Detected 67 (0.84 %) cold users.
RP3betaRecommender: URM Detected 2429 (9.35 %) cold items.
ItemKNNCFRecommender: URM Detected 67 (0.84 %) cold users.
ItemKNNCFRecommender: URM Detected 2429 (9.35 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
IALSRecommender: URM Detected 67 (0.84 %) cold users.
IALSRecommender: URM Detected 2429 (9.35 %) cold items.
ItemKNNCBFRecommender: URM Detected 67 (0.84 %) cold users.
ItemKNNCBFRecommender: URM Detected 2429 (9.35 %) cold items.
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Epoch 1 of 254. Elapsed time 0.02 sec
SLIM_BPR_Recommender: Epoch 2 of 254. Elapsed time 0.04 sec
SLIM_BPR_Recommender: Epoch 3 of 254. Elapsed time 0.08 sec
SLIM_BPR_Recommender: Epoch 4 of 254. Elapsed time 0.12 sec
SLIM_

SLIM_BPR_Recommender: Epoch 123 of 254. Elapsed time 3.05 sec
SLIM_BPR_Recommender: Epoch 124 of 254. Elapsed time 3.08 sec
SLIM_BPR_Recommender: Epoch 125 of 254. Elapsed time 3.10 sec
SLIM_BPR_Recommender: Epoch 126 of 254. Elapsed time 3.12 sec
SLIM_BPR_Recommender: Epoch 127 of 254. Elapsed time 3.14 sec
SLIM_BPR_Recommender: Epoch 128 of 254. Elapsed time 3.17 sec
SLIM_BPR_Recommender: Epoch 129 of 254. Elapsed time 3.19 sec
SLIM_BPR_Recommender: Epoch 130 of 254. Elapsed time 3.21 sec
SLIM_BPR_Recommender: Epoch 131 of 254. Elapsed time 3.23 sec
SLIM_BPR_Recommender: Epoch 132 of 254. Elapsed time 3.25 sec
SLIM_BPR_Recommender: Epoch 133 of 254. Elapsed time 3.27 sec
SLIM_BPR_Recommender: Epoch 134 of 254. Elapsed time 3.29 sec
SLIM_BPR_Recommender: Epoch 135 of 254. Elapsed time 3.31 sec
SLIM_BPR_Recommender: Epoch 136 of 254. Elapsed time 3.33 sec
SLIM_BPR_Recommender: Epoch 137 of 254. Elapsed time 3.36 sec
SLIM_BPR_Recommender: Epoch 138 of 254. Elapsed time 3.38 sec
SLIM_BPR

Deallocating Cython objects
Similarity column 25975 ( 100 % ), 4746.94 column/sec, elapsed time 0.09 min
Similarity column 25975 ( 100 % ), 2720.09 column/sec, elapsed time 0.16 min
IALSRecommender: Epoch 1 of 45. Elapsed time 4.43 sec
IALSRecommender: Epoch 2 of 45. Elapsed time 8.53 sec
IALSRecommender: Epoch 3 of 45. Elapsed time 12.59 sec
IALSRecommender: Epoch 4 of 45. Elapsed time 17.32 sec
IALSRecommender: Epoch 5 of 45. Elapsed time 21.40 sec
IALSRecommender: Epoch 6 of 45. Elapsed time 25.41 sec
IALSRecommender: Epoch 7 of 45. Elapsed time 29.89 sec
IALSRecommender: Epoch 8 of 45. Elapsed time 34.49 sec
IALSRecommender: Epoch 9 of 45. Elapsed time 38.66 sec
IALSRecommender: Epoch 10 of 45. Elapsed time 42.98 sec
IALSRecommender: Epoch 11 of 45. Elapsed time 47.53 sec
IALSRecommender: Epoch 12 of 45. Elapsed time 52.16 sec
IALSRecommender: Epoch 13 of 45. Elapsed time 56.50 sec
IALSRecommender: Epoch 14 of 45. Elapsed time 1.01 min
IALSRecommender: Epoch 15 of 45. Elapsed time 

In [14]:
#check num 1
#Config 622
recommender=WeightedHybridV3V4forBayesianSearch(URM_train,ICM_all,recsToFit)
best_parameters={'w0': 0.7771253987559757, 'w1': 2.283910304115099, 'w2': 10.0, 'w3': 1.8063059091826554, 'w4': 6.234284335332363, 'w5': 6.4128417124725345}
recommender.fit(**best_parameters)

recommender2=WeightedHybridV3V4forBayesianSearch(URM_train2,ICM_all,recsToFit2)
best_parameters={'w0': 0.7771253987559757, 'w1': 2.283910304115099, 'w2': 10.0, 'w3': 1.8063059091826554, 'w4': 6.234284335332363, 'w5': 6.4128417124725345}
recommender2.fit(**best_parameters)

recommender3=WeightedHybridV3V4forBayesianSearch(URM_train3,ICM_all,recsToFit3)
best_parameters={'w0': 0.7771253987559757, 'w1': 2.283910304115099, 'w2': 10.0, 'w3': 1.8063059091826554, 'w4': 6.234284335332363, 'w5': 6.4128417124725345}
recommender3.fit(**best_parameters)

WeightedHybridV3V4forBayesianSearch: URM Detected 73 (0.92 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2431 (9.36 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 58 (0.73 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2418 (9.31 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 67 (0.84 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2429 (9.35 %) cold items.
--------FITTING...-------


In [16]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender)
MAP = result_dict[10]["MAP"]
#print("MAP  recommeder test 1: "+str(MAP)+"\n")

result_dict, _ = evaluator_test2.evaluateRecommender(recommender2)
MAP2 = result_dict[10]["MAP"]
#print("MAP  recommeder test 2: "+str(MAP2)+"\n")

result_dict, _ = evaluator_test3.evaluateRecommender(recommender3)
MAP3 = result_dict[10]["MAP"]
#print("MAP  recommeder test 2: "+str(MAP3)+"\n")

mappozzo=(MAP+MAP2+MAP3)/3
print("\nAVERAGE: ["+str(mappozzo)+"]")

EvaluatorHoldout: Processed 5656 ( 100.00% ) in 15.74 sec. Users per second: 359
MAP  recommeder test 1: 0.07399989928131133

EvaluatorHoldout: Processed 5662 ( 100.00% ) in 15.48 sec. Users per second: 366
MAP  recommeder test 2: 0.07816959888669146

EvaluatorHoldout: Processed 5680 ( 100.00% ) in 16.01 sec. Users per second: 355
MAP  recommeder test 2: 0.07586226632834922


AVERAGE: [0.07601058816545067]


In [17]:
#check num 2 0.073 on validation
#config 1252
recommender=WeightedHybridV3V4forBayesianSearch(URM_train,ICM_all,recsToFit)
best_parameters={'w0': 0.9291195521014475, 'w1': 2.9323133970481736, 'w2': 9.20584043071056, 'w3': 2.398420310624778, 'w4': 7.371572182380706, 'w5': 7.741397895020033}
recommender.fit(**best_parameters)

recommender2=WeightedHybridV3V4forBayesianSearch(URM_train2,ICM_all,recsToFit2)
best_parameters={'w0': 0.9291195521014475, 'w1': 2.9323133970481736, 'w2': 9.20584043071056, 'w3': 2.398420310624778, 'w4': 7.371572182380706, 'w5': 7.741397895020033}
recommender2.fit(**best_parameters)

recommender3=WeightedHybridV3V4forBayesianSearch(URM_train3,ICM_all,recsToFit3)
best_parameters={'w0': 0.9291195521014475, 'w1': 2.9323133970481736, 'w2': 9.20584043071056, 'w3': 2.398420310624778, 'w4': 7.371572182380706, 'w5': 7.741397895020033}
recommender3.fit(**best_parameters)

WeightedHybridV3V4forBayesianSearch: URM Detected 73 (0.92 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2431 (9.36 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 58 (0.73 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2418 (9.31 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 67 (0.84 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2429 (9.35 %) cold items.
--------FITTING...-------


In [18]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender)
MAP = result_dict[10]["MAP"]
#print("MAP  recommeder test 1: "+str(MAP)+"\n")
result_dict, _ = evaluator_test2.evaluateRecommender(recommender2)
MAP2 = result_dict[10]["MAP"]
#print("MAP  recommeder test 2: "+str(MAP2)+"\n")


result_dict, _ = evaluator_test3.evaluateRecommender(recommender3)
MAP3 = result_dict[10]["MAP"]
#print("MAP  recommeder test 2: "+str(MAP3)+"\n")

mappozzo=(MAP+MAP2+MAP3)/3
print("\nAVERAGE: ["+str(mappozzo)+"]")

EvaluatorHoldout: Processed 5656 ( 100.00% ) in 15.65 sec. Users per second: 361
MAP  recommeder test 1: 0.07417012870454105

EvaluatorHoldout: Processed 5662 ( 100.00% ) in 15.58 sec. Users per second: 363
MAP  recommeder test 2: 0.07850470843157566

EvaluatorHoldout: Processed 5680 ( 100.00% ) in 16.76 sec. Users per second: 339
MAP  recommeder test 2: 0.07601762146401331


AVERAGE: [0.07623081953337667]


In [19]:
#check num 3, 0.073 on validation
#config 1257 should be best ever
recommender=WeightedHybridV3V4forBayesianSearch(URM_train,ICM_all,recsToFit)
best_parameters={'w0': 0.7814026405172689, 'w1': 2.9739152141760856, 'w2': 9.282586619873573, 'w3': 2.4611257593408284, 'w4': 7.584599871341396, 'w5': 7.67927578225758}
recommender.fit(**best_parameters)

recommender2=WeightedHybridV3V4forBayesianSearch(URM_train2,ICM_all,recsToFit2)
best_parameters={'w0': 0.7814026405172689, 'w1': 2.9739152141760856, 'w2': 9.282586619873573, 'w3': 2.4611257593408284, 'w4': 7.584599871341396, 'w5': 7.67927578225758}
recommender2.fit(**best_parameters)

recommender3=WeightedHybridV3V4forBayesianSearch(URM_train3,ICM_all,recsToFit3)
best_parameters={'w0': 0.7814026405172689, 'w1': 2.9739152141760856, 'w2': 9.282586619873573, 'w3': 2.4611257593408284, 'w4': 7.584599871341396, 'w5': 7.67927578225758}
recommender3.fit(**best_parameters)

WeightedHybridV3V4forBayesianSearch: URM Detected 73 (0.92 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2431 (9.36 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 58 (0.73 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2418 (9.31 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 67 (0.84 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2429 (9.35 %) cold items.
--------FITTING...-------


In [20]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender)
MAP = result_dict[10]["MAP"]
#print("MAP  recommeder test 1: "+str(MAP)+"\n")
result_dict, _ = evaluator_test2.evaluateRecommender(recommender2)
MAP2 = result_dict[10]["MAP"]
#print("MAP  recommeder test 2: "+str(MAP2)+"\n")

result_dict, _ = evaluator_test3.evaluateRecommender(recommender3)
MAP3 = result_dict[10]["MAP"]
#print("MAP  recommeder test 2: "+str(MAP3)+"\n")

mappozzo=(MAP+MAP2+MAP3)/3
print("\nAVERAGE: ["+str(mappozzo)+"]")

EvaluatorHoldout: Processed 5656 ( 100.00% ) in 14.92 sec. Users per second: 379
MAP  recommeder test 1: 0.07401475921351033

EvaluatorHoldout: Processed 5662 ( 100.00% ) in 15.98 sec. Users per second: 354
MAP  recommeder test 2: 0.07840869448731903

EvaluatorHoldout: Processed 5680 ( 100.00% ) in 15.41 sec. Users per second: 369
MAP  recommeder test 2: 0.07594194418605982


AVERAGE: [0.07612179929562973]


In [21]:

#--------!!!!! QUESTO E' QUELLO CON MAP 0.091 SU KAGGLE !!!!!!!!!!!!!!!!--------


#check num 4 config 725
#validation MAP: 0.0702271
#test MAP: 0.0649211
recommender=WeightedHybridV3V4forBayesianSearch(URM_train,ICM_all,recsToFit)
best_parameters={'w0': 0.6464719543114522, 'w1': 3.0330662257244967, 'w2': 10.0, 'w3': 2.5929646423942043, 'w4': 7.558045116812191, 'w5': 7.4137901570719755}
recommender.fit(**best_parameters)

recommender2=WeightedHybridV3V4forBayesianSearch(URM_train2,ICM_all,recsToFit2)
best_parameters={'w0': 0.6464719543114522, 'w1': 3.0330662257244967, 'w2': 10.0, 'w3': 2.5929646423942043, 'w4': 7.558045116812191, 'w5': 7.4137901570719755}
recommender2.fit(**best_parameters)

recommender3=WeightedHybridV3V4forBayesianSearch(URM_train3,ICM_all,recsToFit3)
best_parameters={'w0': 0.6464719543114522, 'w1': 3.0330662257244967, 'w2': 10.0, 'w3': 2.5929646423942043, 'w4': 7.558045116812191, 'w5': 7.4137901570719755}
recommender3.fit(**best_parameters)

WeightedHybridV3V4forBayesianSearch: URM Detected 73 (0.92 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2431 (9.36 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 58 (0.73 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2418 (9.31 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 67 (0.84 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2429 (9.35 %) cold items.
--------FITTING...-------


In [22]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender)
MAP = result_dict[10]["MAP"]
#print("MAP  recommeder test 1: "+str(MAP)+"\n")
result_dict, _ = evaluator_test2.evaluateRecommender(recommender2)
MAP2 = result_dict[10]["MAP"]
#print("MAP  recommeder test 2: "+str(MAP2)+"\n")

result_dict, _ = evaluator_test3.evaluateRecommender(recommender3)
MAP3 = result_dict[10]["MAP"]
#print("MAP  recommeder test 2: "+str(MAP3)+"\n")

mappozzo=(MAP+MAP2+MAP3)/3
print("\nAVERAGE: ["+str(mappozzo)+"]")

EvaluatorHoldout: Processed 5656 ( 100.00% ) in 14.88 sec. Users per second: 380
MAP  recommeder test 1: 0.07438775925039447

EvaluatorHoldout: Processed 5662 ( 100.00% ) in 14.87 sec. Users per second: 381
MAP  recommeder test 2: 0.07820063724169618

EvaluatorHoldout: Processed 5680 ( 100.00% ) in 15.59 sec. Users per second: 364
MAP  recommeder test 2: 0.07605121425200215


AVERAGE: [0.07621320358136427]


## rifitto tutto ecc x kaggle

In [23]:
#devo rifittare con nuovo train/test split

userknn = {}
userknn["topK"] = 102
userknn["shrink"] = 1

itemknn = {}
itemknn["topK"] = 141
itemknn["shrink"] = 47

cy = {}
cy["epochs"] = 254
cy["topK"] = 881
cy["positive_threshold_BPR"] = 0.9765
cy["learning_rate"] = 0.0002
cy["batch_size"] = 10
cy["sgd_mode"] = "sdg"

uicm = {}
uicm["topK"] = 181
uicm["shrink"] = 0.1
uicm["normalize"] = True

iicm = {}
iicm["topK"] = 893
iicm["shrink"] = 2
iicm["normalize"] = True

alpha = {}
alpha["alpha"] = 0.547615508822564
alpha["topK"] = 500

beta= {}
beta["topK"]=500
beta["alpha"]=0.3784740936494376
beta["beta"]=0.1
beta["implicit"]=False
beta["normalize_similarity"]=False

ials={}
ials["epochs"]=45
ials["num_factors"]=50
ials["alpha"]=10.0
ials["epsilon"]=10.0
ials["reg"]=1e-07
ials["init_mean"]=1.0
ials["init_std"]=0.6773548197826565

hybridv2={}
#bayesianSearchhybridV2
#recommenders usati: [alpha,beta,userknn,uicm,cy,itemknn,iicm]
hybridv2["w0"]=0.001
hybridv2["w1"]=0.5593506236737317
hybridv2["w2"]=0.23267769028384777
hybridv2["w3"]=0.10077656126622785
hybridv2["w4"]=0.0001
hybridv2["w5"]=0.0001
hybridv2["w6"]=0.8397993296794579

itemcbf={}
itemcbf["topK"]=107
itemcbf["shrink"]=18.384308506219117
itemcbf["normalize"] = True
itemcbf["feature_weighting"] = 'BM25'
itemcbf["similarity"] = 'jaccard'

# 9 recomm
fits = [alpha,beta,cy,iicm,ials,itemcbf]

recs = [
    P3alphaRecommender,
    RP3betaRecommender,
    #UserKNNCFRecommender,
    #UserIcmKNNCFRecommender,
    SLIM_BPR_Cython,
    #ItemKNNCFRecommender,
    ItemIcmKNNCFRecommender,
    IALSRecommender,
    ItemKNNCBFRecommender
    ]




only_URM = {
        "URM_train" : URM_all
    }

cython = {
        "URM_train" : URM_all,
        "recompile_cython" : False,
        "verbose" : False
    }

also_ICM = {
        "URM_train" : URM_all,
        "ICM" : ICM_all
    }

CBF = {
        "URM_train" : URM_all,
        "ICM_train" : ICM_all
    }

inits = [
        only_URM,only_URM,cython,also_ICM,only_URM,CBF 
    ]

In [ ]:
recsToFit = []
for rec, init in zip(recs, inits):
    recsToFit.append(rec(**init))
for rec, fit in zip(recsToFit, fits):
    rec.fit(**fit)

P3alphaRecommender: URM Detected 1079 (4.15 %) cold items.
RP3betaRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCFRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
IALSRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCBFRecommender: URM Detected 1079 (4.15 %) cold items.
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Epoch 1 of 254. Elapsed time 0.03 sec
SLIM_BPR_Recommender: Epoch 2 of 254. Elapsed time 0.11 sec
SLIM_BPR_Recommender: Epoch 3 of 254. Elapsed time 0.17 sec
SLIM_BPR_Recommender: Epoch 4 of 254. Elapsed time 0.22 sec
SLIM_BPR_Recommender: Epoch 5 of 254. Elapsed time 0.25 sec
SLIM_BPR_Recommender: Epoch 6 of 254. Elapsed time 0.29 sec
SLIM_BPR_Recommender: Epoch 7 of 254. Elapsed time 0.32 sec
SLIM_BPR_Recommender: Epoch 8 of 254. Elapsed time 0.35 sec
SLIM_BPR_Recommender: Epoch 9 of 254. Elapsed time 0

SLIM_BPR_Recommender: Epoch 128 of 254. Elapsed time 3.50 sec
SLIM_BPR_Recommender: Epoch 129 of 254. Elapsed time 3.52 sec
SLIM_BPR_Recommender: Epoch 130 of 254. Elapsed time 3.54 sec
SLIM_BPR_Recommender: Epoch 131 of 254. Elapsed time 3.56 sec
SLIM_BPR_Recommender: Epoch 132 of 254. Elapsed time 3.58 sec
SLIM_BPR_Recommender: Epoch 133 of 254. Elapsed time 3.60 sec
SLIM_BPR_Recommender: Epoch 134 of 254. Elapsed time 3.63 sec
SLIM_BPR_Recommender: Epoch 135 of 254. Elapsed time 3.65 sec
SLIM_BPR_Recommender: Epoch 136 of 254. Elapsed time 3.67 sec
SLIM_BPR_Recommender: Epoch 137 of 254. Elapsed time 3.70 sec
SLIM_BPR_Recommender: Epoch 138 of 254. Elapsed time 3.72 sec
SLIM_BPR_Recommender: Epoch 139 of 254. Elapsed time 3.74 sec
SLIM_BPR_Recommender: Epoch 140 of 254. Elapsed time 3.76 sec
SLIM_BPR_Recommender: Epoch 141 of 254. Elapsed time 3.79 sec
SLIM_BPR_Recommender: Epoch 142 of 254. Elapsed time 3.81 sec
SLIM_BPR_Recommender: Epoch 143 of 254. Elapsed time 3.83 sec
SLIM_BPR

IALSRecommender: Epoch 3 of 45. Elapsed time 12.95 sec
IALSRecommender: Epoch 4 of 45. Elapsed time 18.45 sec
IALSRecommender: Epoch 5 of 45. Elapsed time 25.33 sec
IALSRecommender: Epoch 6 of 45. Elapsed time 29.76 sec
IALSRecommender: Epoch 7 of 45. Elapsed time 35.72 sec
IALSRecommender: Epoch 8 of 45. Elapsed time 41.59 sec
IALSRecommender: Epoch 9 of 45. Elapsed time 46.64 sec
IALSRecommender: Epoch 10 of 45. Elapsed time 51.59 sec
IALSRecommender: Epoch 11 of 45. Elapsed time 55.85 sec
IALSRecommender: Epoch 12 of 45. Elapsed time 1.00 min
IALSRecommender: Epoch 13 of 45. Elapsed time 1.08 min
IALSRecommender: Epoch 14 of 45. Elapsed time 1.16 min
IALSRecommender: Epoch 15 of 45. Elapsed time 1.24 min
IALSRecommender: Epoch 16 of 45. Elapsed time 1.31 min
IALSRecommender: Epoch 17 of 45. Elapsed time 1.39 min
IALSRecommender: Epoch 18 of 45. Elapsed time 1.46 min
IALSRecommender: Epoch 19 of 45. Elapsed time 1.53 min
IALSRecommender: Epoch 20 of 45. Elapsed time 1.61 min
IALSReco

In [ ]:
recommender=WeightedHybridV3V4forBayesianSearch(URM_all,ICM_all,recsToFit)
best_parameters={}
recommender.fit(**best_parameters)

In [ ]:
import time
URM_file = open(r"data_target_users_test.csv", 'r')

URM_file.seek(0)
URM_tuples = []
URM_file.readline()#questa è importante di cristo c'è riga di introduzione
for line in URM_file:
    URM_tuples.append(int(line.replace("\n","")))

usersToPred = np.array(URM_tuples)

start_time = time.time()#BUONE METRICHE PER IL TEMPO QUI
import csv

with open('OUTPUTS\outputIbridoSuperTuning9algos.csv', mode='w',newline='') as recFile:
    
    recFile = csv.writer(recFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    recFile.writerow(['user_id', 'item_list'])
    recs=recommender.recommend(usersToPred, cutoff=10)
    for user_id,rec in zip(usersToPred,recs):
        rec_string =' '.join(map(str, rec[:10]))
        recFile.writerow([user_id, rec_string])
    
end_time = time.time()
print("Ha finito in questo tempo: "+str(end_time-start_time)+" secondi")